In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import seaborn as sns
from collections import defaultdict
import logging
logging.basicConfig()

from diamond.glms.logistic import LogisticRegression

%matplotlib inline

This dataset is from the UCI Machine Learning Repository https://archive.ics.uci.edu/ml/datasets/Soybean+(Small)

Origin: 

Michalski,R.S. 
Learning by being told and learning from examples: an experimental comparison of the two methodes of knowledge acquisition in the context of developing an expert system for soybean desease diagnoiss", 
International Journal of Policy Analysis and Information Systems, 1980, 4(2), 125-161. 

The example here is not intended to be statistically useful, but rather just to show the nuts and bolts of fitting a model using Diamond.

# Load the data and the covariance structure

In [2]:
df = pd.read_csv('soybean_small.csv')
df.head()

,date,plant_stand,precip,temp,hail,crop_hist,area_damaged,severity,seed_tmt,germination,...,sclerotia,fruit_pods,fruit spots,seed,mold_growth,seed_discolor,seed_size,shriveling,roots,NA
0,4,0,2,1,1,1,0,1,0,2,...,0,0,4,0,0,0,0,0,0,D1
1,5,0,2,1,0,3,1,1,1,2,...,0,0,4,0,0,0,0,0,0,D1
2,3,0,2,1,0,2,0,2,1,1,...,0,0,4,0,0,0,0,0,0,D1
3,6,0,2,1,0,1,1,1,0,0,...,0,0,4,0,0,0,0,0,0,D1
4,4,0,2,1,0,3,0,2,0,2,...,0,0,4,0,0,0,0,0,0,D1


Note that the covariance structure, here referred to as the priors, is only for the random effect components. The fixed effects are not regularized in Diamond. The priors data frame has the following columns:
  - grp: the grouping factor (in this case "leaves") that the variables belong to
  - var1: variable in the model which belongs to group factor `grp`
  - var2: same as var1. Set to NaN to denote the variance of var1
  - vcov: (co)variance between var1 and var2
  - sdcor: square root of vcov (optional - not used in Diamond)

In [3]:
priors = pd.read_csv("soybean_small_priors.csv")
priors.head()

,grp,var1,var2,vcov,sdcor
0,leaves,intercept,NaN,0.012599,0.112246
1,leaves,precip,NaN,0.049492,0.222468


# Fit the model

In [4]:
model = LogisticRegression(df, priors)

In [5]:
formula = 'fruiting_bodies ~ 1 + precip + (1 + precip | leaves)'

In [6]:
effects = model.fit(formula, tol=1e-4, verbose=True)

INFO:diamond:creating main design matrix
INFO:diamond:creating leaves design matrix
INFO:diamond:creating covariance matrix
INFO:diamond:creating Hessians
INFO:diamond:creating H_inter for leaves
INFO:diamond:time elapsed: 0.0
INFO:diamond:blocks inverted: 0 of 2
INFO:diamond:creating H_invs


loss:  477.033640854
penalty:  1.90103465679
seconds elapsed: 0
iteration:  0 relative coef change:  1.0 obj:  478.934675511
loss:  471.294137883
penalty:  1.32038267558
seconds elapsed: 0
iteration:  1 relative coef change:  0.356817154498 obj:  472.614520558
loss:  457.603613099
penalty:  1.1549274814
seconds elapsed: 0
iteration:  2 relative coef change:  0.262887394611 obj:  458.75854058
loss:  452.700500964
penalty:  1.12710527809
seconds elapsed: 0
iteration:  3 relative coef change:  0.166151724338 obj:  453.827606242
loss:  450.480230922
penalty:  1.13544067991
seconds elapsed: 0
iteration:  4 relative coef change:  0.101623981243 obj:  451.615671602
loss:  450.207392221
penalty:  1.14885106683
seconds elapsed: 0
iteration:  5 relative coef change:  0.0580275803276 obj:  451.356243288
loss:  450.94455974
penalty:  1.16270077546
seconds elapsed: 0
iteration:  6 relative coef change:  0.0420669036814 obj:  452.107260515
loss:  463.742248907
penalty:  1.17067949821
seconds elapsed

INFO:diamond:extracting coefficients


 520.061868644
penalty:  1.32745581912
seconds elapsed: 0
iteration:  97 relative coef change:  0.000174839764281 obj:  521.389324463
loss:  519.40544099
penalty:  1.32766829512
seconds elapsed: 0
iteration:  98 relative coef change:  0.00105671485457 obj:  520.733109285
loss:  520.18511945
penalty:  1.32772164809
seconds elapsed: 0
iteration:  99 relative coef change:  6.42162644653e-05 obj:  521.512841098
total seconds elapsed: 0
reached convergence after  99  steps. relative coef change:  6.42162644653e-05


# Combine random and fixed effects

In [7]:
results = model.results_dict['leaves'].copy()

In [8]:
for var in model.results_dict["fixed_effects"].variable.unique():
    if var in results.columns:
        results[var] = (results[var] + 
            model.results_dict['fixed_effects'][model.results_dict['fixed_effects'].variable == var].value.values[0])
    else:
        results[var] = (
            model.results_dict['fixed_effects'][model.results_dict['fixed_effects'].variable == var].value.values[0])

In [9]:
results

,leaves,intercept,precip
0,0,-14.371869,6.485837
1,1,-14.327154,6.837357
